# Xorshift Random Number Generator in Sagemath/Jupyter, @iwaokimura

G. Marsagliaの乱数生成器xorshiftを，sagemathで試てみる．この乱数は，2元体上のN次元ベクトル空間Vのある元xを初期値として選び，V上の線形変換Tをxに繰り返し施した値 T^m(x) を生成した乱数値とするものである．

2元体での演算がxorと同じであることにまず注意しよう．ビット列についても同様に，ビット列のxorと，2元体上のベクトル空間の成分の和が同じものになる．更に，ビット列の左，右シフトが，2元体上のベクトル空間の線形変換L, R（後述）になる．以上から，線形変換Tとして，次のような特別なものに注目することを思いつく：
Tを，a, b, cを適切に選んで，T=(1+L^a)(1+R^b)(1+L^c)とする．「適切に」というのは，Tの（V上の線形変換としての）位数が十分大きくなるように（正確には，Tの位数が 2^N-1 になるように），という意味である．

以上の詳細については，「Google Chromeが採用した、擬似乱数生成アルゴリズム「xorshift」の数理」https://blog.visvirial.com/articles/575 が大変読みやすい解説である．

以下では，N=32として，Tの位数が 2^N-1 となるa, b, cの探索を行う．これは，G. Marsaglia, Xorshift RNGs, http://dx.doi.org/10.18637/jss.v008.i14 のはじめの2ページほどに相当する．

In [1]:
N=32; # 次元

In [2]:
G = GL(N, FiniteField(2)); G # 2元体上の，N 次正則（可逆）行列の群

General Linear Group of degree 32 over Finite Field of size 2

In [3]:
G.order() # 試しに G の位数を計算する．

51915237608802545834650365775747737098229791081502830361375666998864635954655274189188618336673413249941113660054889860892524028523586055095785930107241175599128831839106952836064314134436759863399605193683057244940843342962051291562930060275608907956668714644423971391788961835934547494654875533312000000000

In [4]:
G.order().factor() # 素因数分解してみる

2^496 * 3^22 * 5^9 * 7^11 * 11^3 * 13^2 * 17^4 * 19 * 23^2 * 29 * 31^6 * 41 * 43^2 * 47 * 73^3 * 89^2 * 113 * 127^4 * 151^2 * 233 * 241 * 257^2 * 331 * 337 * 601 * 683 * 1103 * 1801 * 2089 * 2731 * 8191^2 * 65537 * 131071 * 178481 * 262657 * 524287 * 2147483647

まず，左シフトに相当する行列 L の成分を与える関数を用意する．(1,2), (2, 3), ..., (31, 32)成分が1, あとはすべて0である行列である．つまり，i行j列の成分が，i+1 == j のとき1, それ以外の時 0 である．ただし，sagemathでは行列，ベクトルは0番目の成分から数えることに注意する．

In [5]:
def Lcoeff(i, j):
    if i+1 == j:
        return FiniteField(2).one()
    else:
        return FiniteField(2).zero()

試しに数字を列挙してみる：

In [6]:
for i in range(0, 32):
    for j in range(0, 32):
        print(i, j, Lcoeff(i,j))

(0, 0, 0)
(0, 1, 1)
(0, 2, 0)
(0, 3, 0)
(0, 4, 0)
(0, 5, 0)
(0, 6, 0)
(0, 7, 0)
(0, 8, 0)
(0, 9, 0)
(0, 10, 0)
(0, 11, 0)
(0, 12, 0)
(0, 13, 0)
(0, 14, 0)
(0, 15, 0)
(0, 16, 0)
(0, 17, 0)
(0, 18, 0)
(0, 19, 0)
(0, 20, 0)
(0, 21, 0)
(0, 22, 0)
(0, 23, 0)
(0, 24, 0)
(0, 25, 0)
(0, 26, 0)
(0, 27, 0)
(0, 28, 0)
(0, 29, 0)
(0, 30, 0)
(0, 31, 0)
(1, 0, 0)
(1, 1, 0)
(1, 2, 1)
(1, 3, 0)
(1, 4, 0)
(1, 5, 0)
(1, 6, 0)
(1, 7, 0)
(1, 8, 0)
(1, 9, 0)
(1, 10, 0)
(1, 11, 0)
(1, 12, 0)
(1, 13, 0)
(1, 14, 0)
(1, 15, 0)
(1, 16, 0)
(1, 17, 0)
(1, 18, 0)
(1, 19, 0)
(1, 20, 0)
(1, 21, 0)
(1, 22, 0)
(1, 23, 0)
(1, 24, 0)
(1, 25, 0)
(1, 26, 0)
(1, 27, 0)
(1, 28, 0)
(1, 29, 0)
(1, 30, 0)
(1, 31, 0)
(2, 0, 0)
(2, 1, 0)
(2, 2, 0)
(2, 3, 1)
(2, 4, 0)
(2, 5, 0)
(2, 6, 0)
(2, 7, 0)
(2, 8, 0)
(2, 9, 0)
(2, 10, 0)
(2, 11, 0)
(2, 12, 0)
(2, 13, 0)
(2, 14, 0)
(2, 15, 0)
(2, 16, 0)
(2, 17, 0)
(2, 18, 0)
(2, 19, 0)
(2, 20, 0)
(2, 21, 0)
(2, 22, 0)
(2, 23, 0)
(2, 24, 0)
(2, 25, 0)
(2, 26, 0)
(2, 27, 0)
(2, 28, 0)
(2, 29,

上の関数 Lcoeff() を元に，左シフトに対応する行列Lを作る：

In [7]:
L = matrix([[Lcoeff(i,j) for j in range(32)] for i in range(32)])

遅ればせながらここで単位行列を用意する：

In [8]:
idelem = matrix.identity(FiniteField(2), N) # id element

以下，右シフトについて同様のことをする．

In [9]:
def Rcoeff(i, j):
    if i == j+1:
        return FiniteField(2).one()
    else:
        return FiniteField(2).zero()

In [10]:
R = matrix([[Rcoeff(i,j) for j in range(32)] for i in range(32)])

あとは，T=(1+L^s)(1+R^t)(1+L^u) の位数を，s < u の範囲でループを回しながら計算し，2^N-1になるようなs, t, uを列挙するだけ．

In [13]:
@parallel(p_iter='multiprocessing', ncpus=4)
def make_list():
    for s in range(1,31):
        for t in range(1, 31):
            for u in range(s, 31):
                if ((idelem+L^s)*(idelem+R^t)*(idelem+L^u)).multiplicative_order() == 2^N-1:
                    print(s, t, u)

In [14]:
make_list()

(1, 3, 10)
(1, 5, 16)
(1, 5, 19)
(1, 9, 29)
(1, 11, 6)
(1, 11, 16)
(1, 19, 3)
(1, 21, 20)


KeyboardInterrupt: 

以上．